With current VAE model, it works but not impressively well. (lacks good generalisation)

* Load model

In [69]:
import os
import sys
import torch
from torchvision import transforms
import numpy as np
from torchvision.utils import save_image

In [4]:
sys.path.append('../')

In [5]:
from src.models.vae import VAE
from src.data.dataset import CityImageDataset
from src.data.invert import Invert
from settings import FIGPATH
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cpu


* Load model

In [8]:
image_channels = 1
model = VAE(image_channels=image_channels, h_dim=1024).to(device)
model.load_state_dict(torch.load('../src/models/vae.torch', map_location='cpu'))  

* Prep data

In [9]:
datapath = os.path.join(FIGPATH,'figures_20000')

In [21]:
train_dataset = CityImageDataset(root=os.path.join(datapath,'train'),transform=transforms.Compose([
    transforms.Grayscale(),
    Invert(),
    transforms.CenterCrop(128),    # instead of random crop
    transforms.Resize(64),
    transforms.ToTensor(),
]))
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=1,
    shuffle = True
)

* Interpolate

In [70]:
idx1 = 129
idx2 = 250
npoints = 10
points = []
with torch.no_grad():
    z1 = model.encode(train_loader.dataset[idx1][0].unsqueeze(0))[0].numpy()
    z2 = model.encode(train_loader.dataset[idx2][0].unsqueeze(0))[0].numpy()
    
    # Interpolation
    diff = z2-z1
    for i in range(0,npoints+1):
        point = z1 + i*diff/npoints
        points.append(point)
        
    # Plot each interpolated point
    for i,point in enumerate(points):
        image_tensor = model.decode(torch.from_numpy(point))
        save_image(image_tensor.cpu(),"test%s.png" % i)